In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 952.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
from datasets.load import load_from_disk
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.optim as optim
from tqdm.notebook import tqdm
import torch.nn.functional as F
import csv
from transformers import TextDataset
from datasets import load_dataset, Dataset, concatenate_datasets, ClassLabel, Features, Value

# # Set the device
# if torch.cuda.is_available():    
#     device = torch.device("cuda")
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
#     print('We will use the GPU:', torch.cuda.get_device_name(0))
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load MRPC dataset
MRPC = load_dataset("glue", "mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
MRPC

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

# augmentation

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
knowledge_dataset_params = {"MRPC": {"n_iter": 10, "p_mask":0.1, "p_pos": 0.1, "p_ng":0.25}}

In [ ]:
import transformers

# Tokenize the input sentence
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Load word distributions from disk
word_distributions = torch.load('/content/drive/MyDrive/CS685/project/word_distributions.pt')

In [ ]:
# Define a function to replace a single word with a random word of the same part-of-speech tag
def replace_word_with_random_word_of_same_pos(word, pos_tag):
    # Get the unigram word distribution for the given part-of-speech tag
    word_distribution = word_distributions[pos_tag]
    
    # Sample a new word from the distribution
    new_word_index = torch.multinomial(word_distribution, 1).item()
    new_word = tokenizer.convert_ids_to_tokens([new_word_index])[0]
    
    return new_word

In [ ]:
def pos_guided_word_replacement(sentence, word_to_replace):
  # Tokenize the sentence
  tokenized_sentence = tokenizer.tokenize(sentence)

  pos_tag_to_replace = nltk.pos_tag([word_to_replace])[0][1]

  for i in range(len(tokenized_sentence)):
    if tokenized_sentence[i] == word_to_replace:
        return replace_word_with_random_word_of_same_pos(word_to_replace, pos_tag_to_replace)

In [ ]:
cur_train = MRPC['train']

In [ ]:
cur_train

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [ ]:
from collections import defaultdict
pos_tag_word_map = defaultdict(lambda: set())
with torch.no_grad(), tqdm(total=len(cur_train['sentence1']), desc="Computing the word distributions with sentence1") as pbar:
  for s in cur_train['sentence1']:
    for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(s)):
      pos_tag_word_map[pos_tag].add(word)
    pbar.update(1)
with torch.no_grad(), tqdm(total=len(cur_train['sentence2']), desc="Computing the word distributions with sentence2") as pbar:
  for s in cur_train['sentence2']:
    for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(s)):
      pos_tag_word_map[pos_tag].add(word)
    pbar.update(1)

Computing the word distributions with sentence1:   0%|          | 0/3668 [00:00<?, ?it/s]

Computing the word distributions with sentence2:   0%|          | 0/3668 [00:00<?, ?it/s]

In [ ]:
pos_tag_word_map_list = defaultdict(lambda: [])
for pos_tag in pos_tag_word_map:
  pos_tag_word_map_list[pos_tag] = list(pos_tag_word_map[pos_tag])

In [ ]:
import random
data = {}
for column in cur_train.column_names:
  data[column] = []

idx_counter = len(cur_train)

error_count = 0

# Modifying the first sentence
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using first sentence") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['MRPC']['n_iter']):
        res = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence1'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['MRPC']['p_mask']:
            res.append('[MASK]')
          elif knowledge_dataset_params['MRPC']['p_mask'] <= X < knowledge_dataset_params['MRPC']['p_mask']+knowledge_dataset_params['MRPC']['p_pos']:
            res.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['MRPC']['p_ng']:
          synthetic_text1 = ' '.join(random.sample(res, min(len(res), random.randint(1, 5))))
          data['sentence1'].append(synthetic_text1)
          data['sentence2'].append(row['sentence2'])
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

# Modifying the second sentence
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using second sentence") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['MRPC']['n_iter']):
        res = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence2'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['MRPC']['p_mask']:
            res.append('[MASK]')
          elif knowledge_dataset_params['MRPC']['p_mask'] <= X < knowledge_dataset_params['MRPC']['p_mask']+knowledge_dataset_params['MRPC']['p_pos']:
            res.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['MRPC']['p_ng']:
          synthetic_text2 = ' '.join(random.sample(res, min(len(res), random.randint(1, 5))))
          data['sentence1'].append(row['sentence1'])
          data['sentence2'].append(synthetic_text2)
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

# Modifying both sentences
with torch.no_grad(), tqdm(total=len(cur_train), desc="Computing the synthetic examples using both sentences") as pbar:
  for idx, row in enumerate(cur_train):
    try:
      for _ in range(knowledge_dataset_params['MRPC']['n_iter']):
        res1 = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence1'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['MRPC']['p_mask']:
            res1.append('[MASK]')
          elif knowledge_dataset_params['MRPC']['p_mask'] <= X < knowledge_dataset_params['MRPC']['p_mask']+knowledge_dataset_params['MRPC']['p_pos']:
            res1.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res1.append(word)
        res2 = []
        for word, pos_tag in nltk.pos_tag(nltk.word_tokenize(row['sentence2'])):
          X = random.uniform(0,1)
          if X < knowledge_dataset_params['MRPC']['p_mask']:
            res2.append('[MASK]')
          elif knowledge_dataset_params['MRPC']['p_mask'] <= X < knowledge_dataset_params['MRPC']['p_mask']+knowledge_dataset_params['MRPC']['p_pos']:
            res2.append(random.choice(pos_tag_word_map_list[pos_tag]))
          else:
            res2.append(word)
        if random.uniform(0,1) < knowledge_dataset_params['MRPC']['p_ng']:
          synthetic_text1 = ' '.join(random.sample(res1, min(len(res1), random.randint(1, 5))))
          synthetic_text2 = ' '.join(random.sample(res2, min(len(res2), random.randint(1, 5))))
          data['sentence1'].append(synthetic_text1)
          data['sentence2'].append(synthetic_text2)
          data['label'].append(row['label'])
          data['idx'].append(idx_counter)
          idx_counter += 1
          # data['label_text'].append(row['label_text'])
    except Exception as e:
      print(e)
      error_count += 1
    finally:
      pbar.update(1)

print('\nMissed Entries', str(error_count))

Computing the synthetic examples using first sentence:   0%|          | 0/3668 [00:00<?, ?it/s]

Computing the synthetic examples using second sentence:   0%|          | 0/3668 [00:00<?, ?it/s]

Computing the synthetic examples using both sentences:   0%|          | 0/3668 [00:00<?, ?it/s]


Missed Entries 0


In [ ]:
from datasets import Dataset
new_train = Dataset.from_dict(data)

In [ ]:
# schema of MRPC dataset
ds_schema = Features({
    "sentence1": Value("string"),
    "sentence2": Value("string"),
    "label": ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
    "idx": Value(dtype='int32', id=None)
})

new_train = new_train.cast(ds_schema)

Casting the dataset:   0%|          | 0/27380 [00:00<?, ? examples/s]

In [ ]:
new_train = new_train.cast(ds_schema)

Casting the dataset:   0%|          | 0/27380 [00:00<?, ? examples/s]

In [ ]:
cur_train[:10]

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
  'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .',
  'Revenue in the first quarter of the year dropped 15 percent from the same period a year earlier .',
  'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .',
  'The DVD-CCA then appealed to the state Supreme Court .',
  'That compared with $ 35.18 million , or 24 cents per share , in the year-ago period .',
  'Shares of Genentech , a much larger company with several products on the market , ros

In [ ]:
new_train[:10]

{'sentence1': [', brother accused Raymond he',
  '. ``',
  'evidence , deliberately',
  'Yucaipa before',
  '$',
  'They 10 .',
  'cleared . , had',
  'he offering on',
  'high of',
  'Friday [MASK]'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering th

In [ ]:
knowledge_dataset = concatenate_datasets([cur_train, new_train])

In [ ]:
MRPC['train'] = knowledge_dataset

In [ ]:
MRPC.save_to_disk('/content/drive/MyDrive/CS685/project/datasets/MRPC')

Saving the dataset (0/1 shards):   0%|          | 0/31048 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/408 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
MRPC

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 31048
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
MRPC['train'][:10]

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
  'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .',
  'Revenue in the first quarter of the year dropped 15 percent from the same period a year earlier .',
  'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .',
  'The DVD-CCA then appealed to the state Supreme Court .',
  'That compared with $ 35.18 million , or 24 cents per share , in the year-ago period .',
  'Shares of Genentech , a much larger company with several products on the market , ros